<a href="https://colab.research.google.com/github/RoyElkabetz/Text-Summarization-with-Deep-Learning/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## uncomment only if running from google.colab
# clone the git reposetory
!git clone https://github.com/RoyElkabetz/Text-Summarization-with-Deep-Learning
# add path to .py files for import
import sys
sys.path.insert(1, "/content/Text-Summarization-with-Deep-Learning")

Cloning into 'Text-Summarization-with-Deep-Learning'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 53 (delta 28), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [10]:
## uncomment if you want to mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%matplotlib inline
import time
import pandas as pd

import torch
from torchtext.datasets import IMDB
import torchtext.data as data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torch import nn




print(f'torch {torch.__version__}')
print('Device properties:')
if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_data = torch.cuda.get_device_properties(0)
    gpu_name = gpu_data.name
    gpu_mem  = f'{gpu_data.total_memory * 1e-9:.02f} Gb'
    print(f'GPU: {gpu_name}\nMemory: {gpu_mem}')
else:
    device = torch.device("cpu")
    print('CPU')

torch 1.9.0+cu102
Device properties:
CPU


In [5]:
class DataFrameDataset(Dataset):
  """Create a torch.utils.data.Dataset from a pandas.DataFrame or a CSV file."""

  def __init__(self, csv_file_path=None, pd_dataframe=None, only_columns=None):
    """
      Args:
      csv_file_path (string): Path to the csv file with annotations.
      pd_dataframe (Pandas DataFrame): A Pandas DataFrame with containing the
      data.
      only_columns (list): A List of colums names from the data. 
    """
    if isinstance(pd_dataframe, pd.DataFrame):
      self.df = pd_dataframe 
    else:
      self.df = pd.read_csv(csv_file_path)

    if only_columns is not None:
      if isinstance(only_columns, list):
        for item in only_columns:
          if item not in self.df.columns:
            raise ValueError(f"Got a column name '{item}' in only_columns which is not in DataFrame columns.")
        self.only_columns = only_columns
      else:
        raise TypeError(f"only_columns must be a <class 'list'>, instead got a {type(only_columns)}.")
    else:
      self.only_columns = list(self.df.columns)

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    row = self.df.iloc[idx][self.only_columns]
    row_list = [item for item in row]
    return row_list

## Get the IMDB dataset and create a vocabulary from the train dataset
I use the IMDB test data as train

In [6]:
tokenizer = get_tokenizer('basic_english')
train_iter = IMDB(split='test')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>", "<sos>", "<eos>"])
vocab.set_default_index(vocab["<unk>"])

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 70.0MB/s]


## Create text and labels pipelines

In [7]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 0 if x=='neg' else 1

## Print some random samples and the size of the dataset

In [8]:
train_iter = IMDB(split='test')
n_samples = len(train_iter)
random_list = torch.randint(0, n_samples - 1, (4, ))
labels = []
for i, (label, text) in enumerate(train_iter):
    labels.append(label)
    if i in random_list:
        print(f'Label: {label_pipeline(label)}')
        print(f'Text: {text}')
        print(f'Split: {tokenizer(text)}')
        print(f'Tokens: {text_pipeline(text)}\n')
print('Number of classes: {}'.format(len(set(labels))))
print('Number of samples: {}'.format(n_samples))

Label: 0
Text: I say "flick" because this doesn't deserve the appellation "movie", and certainly not "film". I regret paying for the rental, and although I've never walked out on a movie before, this would have been it, had I seen it in a theatre. A society living underground in the future (oooh, THAT'S original), lots of burning barrel drums, unexplained ambient light shining through windows, an ungrateful woman and her shock-muted son...the list goes on and on. C. Thomas Howell affects the husky voice of the stereotypical loner; you know like Eastwood's been done to death. He needs special sunglasses to remember his wife and child, yet in the flashbacks, he's the same age! Talk about a poor memory! I stared incredulously when the little boy Abe randomly pushes a code into a door and it opens! No tension, pithy religious (what religion?) under/overtones...saddest of all: I expected better from Roddy Piper;<br /><br />Quite possibly the worst movie experience in my life.
Split: ['i', '

In [28]:
VALID_DATASET_PATH = '/content/gdrive/MyDrive/Datasets/Text/IMDB_validation_dataset.csv'
TEST_DATASET_PATH = '/content/gdrive/MyDrive/Datasets/Text/IMDB_test_with_summary_dataset.csv'

valid_dataset = DataFrameDataset(csv_file_path=VALID_DATASET_PATH, only_columns=['label', 'text'])
test_dataset = DataFrameDataset(csv_file_path=TEST_DATASET_PATH, only_columns=['label', 'summary'])
full_test_dataset = DataFrameDataset(csv_file_path=TEST_DATASET_PATH, only_columns=['label', 'text', 'summary'])
print(f'Validation dataset size is: {len(valid_dataset)}')
print(f'Test dataset size is: {len(test_dataset)}')

Validation dataset size is: 16434
Test dataset size is: 8467


In [29]:
test_loader = DataLoader(full_test_dataset, batch_size=2, shuffle=False)
for batch in test_loader:
  print('Print samples from a single batch:\n')
  labels, texts, summaries = batch
  for i in range(len(labels)):
    print(f'Sample: {i}')
    print(f'Label: {labels[i]}')
    print(f'Text: {texts[i]}')
    print(f'Label: {summaries[i]}')
    print('\n')
  break

Print samples from a single batch:

Sample: 0
Label: neg
Text: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to maki
Label: I AM CURIOUS-YELLOW is a film about a young Swedish drama student named Lena who wants to learn everything she can about life. The plot is centered around a maki girl named Maki who wants to focus her attentions on the maki boy's maki, which is a game of maki. The film was released in 1967 and has been rated 4/5 (Sweden).


Sample: 1
Label: neg
Text: "I Am Curious: Yellow" is a risible and pretentious steaming pil

In [30]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

In [31]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.activation = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = self.fc(embedded)
        output = self.activation(x)
        return output

In [32]:
train_iter = IMDB(split='test')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [39]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 100
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [42]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 0.001  # learning rate
BATCH_SIZE = 32 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = IMDB(split='test')
train_dataset = to_map_style_dataset(train_iter)
# test_dataset = to_map_style_dataset(test_iter)
# num_train = int(len(train_dataset) * 0.95)
# split_train_, split_valid_ = \
#     random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE,
                              shuffle=False, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=False, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    # if total_accu is not None and total_accu > accu_val:
    #   scheduler.step()
    # else:
    total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   100/  782 batches | accuracy    0.493
| epoch   1 |   200/  782 batches | accuracy    0.498
| epoch   1 |   300/  782 batches | accuracy    0.497
| epoch   1 |   400/  782 batches | accuracy    0.498
| epoch   1 |   500/  782 batches | accuracy    0.500
| epoch   1 |   600/  782 batches | accuracy    0.500
| epoch   1 |   700/  782 batches | accuracy    0.509


KeyboardInterrupt: ignored

In [ ]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.907


In [ ]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news
